In [22]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from corr_parc import calculate_partial_correlation_df, apply_partial_correlation_criteria

In [23]:
df_brent = pd.read_csv("./processed_tables/brent.csv")
df_precios = pd.read_csv("./processed_tables/precios.csv")
df_precipitacion = pd.read_csv("./processed_tables/precipitacion.csv")
df_temperatura = pd.read_csv("./processed_tables/temperatura.csv")
df_trm = pd.read_csv("./processed_tables/TRM.csv")

In [24]:
df_brent = df_brent.rename(columns={"Fecha": "Date"})
df_precipitacion = df_precipitacion.rename(columns={"date": "Date"})
df_temperatura = df_temperatura.rename(columns={"date": "Date"})
df_trm = df_trm.rename(columns={"vigenciadesde": "Date"})

# Perform left joins
df_merged = pd.merge(df_precios, df_brent, on="Date", how="left")
df_merged = pd.merge(df_merged, df_precipitacion, on="Date", how="left")
df_merged = pd.merge(df_merged, df_temperatura, on="Date", how="left")
df_merged = pd.merge(df_merged, df_trm, on="Date", how="left")

Quitamos `precipitacion_unidadmedida`

In [25]:
df_merged.drop("precipitacion_unidadmedida", axis=1, inplace=True)

In [26]:
df_merged.to_csv("processed_tables/merged_no_dams.csv", index=False)

### Correlación variables explicativas con variable dependiente con data estandarizada

In [27]:
scaler = StandardScaler()
exogen_data_df = df_merged.drop(["Date", "energy_price"], axis=1)
exogen_data = scaler.fit_transform(exogen_data_df)
exogen_data_df_standardized = pd.DataFrame(exogen_data, columns=exogen_data_df.columns)
scaled_df_merged = exogen_data_df_standardized.copy()
scaled_df_merged["energy_price"] = df_merged["energy_price"]

## Matriz de correlación

In [28]:
correlation_matrix = exogen_data_df.corr()
print(f"Correlation Matrix Original: {correlation_matrix.shape}")
correlation_matrix.style.background_gradient(cmap="coolwarm")


Correlation Matrix Original: (67, 67)


,brent_value,precipitacion_amazonas,precipitacion_antioquia,precipitacion_arauca,precipitacion_atlantico,precipitacion_bogota,precipitacion_bolivar,precipitacion_boyaca,precipitacion_caldas,precipitacion_caqueta,precipitacion_casanare,precipitacion_cauca,precipitacion_cesar,precipitacion_choco,precipitacion_cordoba,precipitacion_cundinamarca,precipitacion_guainia,precipitacion_guaviare,precipitacion_huila,precipitacion_la guajira,precipitacion_magdalena,precipitacion_meta,precipitacion_narino,precipitacion_norte de santander,precipitacion_putumayo,precipitacion_quindio,precipitacion_risaralda,precipitacion_san andres providencia,precipitacion_santander,precipitacion_sucre,precipitacion_tolima,precipitacion_valle del cauca,precipitacion_vaupes,precipitacion_vichada,temp_AMAZONAS,temp_ANTIOQUIA,temp_ARAUCA,temp_ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA,temp_ATLANTICO,temp_BOGOTA,temp_BOLIVAR,temp_BOYACA,temp_CALDAS,temp_CAQUETA,temp_CASANARE,temp_CAUCA,temp_CESAR,temp_CHOCO,temp_CORDOBA,temp_CUNDINAMARCA,temp_GUAINIA,temp_GUAVIARE,temp_HUILA,temp_LA GUAJIRA,temp_MAGDALENA,temp_META,temp_NARINO,temp_NORTE DE SANTANDER,temp_PUTUMAYO,temp_QUINDIO,temp_RISARALDA,temp_SANTANDER,temp_SUCRE,temp_TOLIMA,temp_VALLE DEL CAUCA,temp_VICHADA,TRM
brent_value,1.000000,0.015180,0.329614,0.034924,0.115467,0.252189,0.191135,0.051210,0.201223,0.051083,0.006283,0.036725,0.204421,-0.312554,0.320371,0.037665,0.134976,0.183881,-0.071111,0.154511,0.358489,0.214289,0.186630,0.126432,-0.153138,0.052736,0.183884,0.134912,0.286426,0.233963,0.020231,0.155809,0.001619,-0.013260,-0.195409,-0.123097,-0.140329,-0.094225,-0.202322,-0.326451,-0.313565,-0.272161,-0.519461,-0.265061,-0.239194,-0.313540,-0.166155,-0.356461,-0.215685,-0.235658,-0.169176,-0.337520,-0.216031,0.040892,-0.125118,0.026172,0.085410,-0.295623,-0.011673,-0.356808,-0.192665,-0.476708,-0.304795,-0.075845,-0.238194,0.032238,-0.317626
precipitacion_amazonas,0.015180,1.000000,-0.000581,-0.066088,-0.075014,0.106463,-0.033369,0.003890,0.029611,-0.022714,-0.013511,0.087176,-0.015286,0.083052,0.000550,0.093379,-0.059546,0.043675,-0.010479,-0.105463,-0.001357,-0.022306,0.036657,0.046852,-0.000844,0.066723,0.028251,-0.062482,0.025980,0.008736,0.045353,0.100075,0.003200,0.008473,-0.232514,0.052096,0.073518,-0.060093,0.110331,0.052482,0.082042,0.051021,-0.056883,0.055474,0.060518,0.024629,0.086051,0.127459,0.070835,0.014092,0.101549,0.026618,0.082148,0.017831,-0.011986,0.122759,0.101894,-0.008471,0.041728,-0.011253,0.026718,-0.007228,0.103934,0.071053,0.037895,-0.016458,-0.011616
precipitacion_antioquia,0.329614,-0.000581,1.000000,0.314739,0.084880,0.270766,0.324677,0.391595,0.594696,0.209265,0.116232,0.335611,0.137814,0.003961,0.455286,0.297180,0.074976,0.351991,0.286523,0.140865,0.404900,0.345804,0.351733,0.194504,0.083891,0.395891,0.384270,0.141533,0.519930,0.306948,0.466152,0.514026,0.164454,0.026048,-0.087057,-0.253362,0.180829,0.004907,-0.234126,-0.196108,-0.288471,-0.101192,-0.380997,-0.220491,-0.184058,-0.299346,-0.264779,-0.193303,-0.319414,-0.303399,-0.125014,-0.405770,-0.185246,0.030130,-0.105091,-0.009625,0.146918,-0.254834,-0.034198,-0.405214,-0.309354,-0.387502,-0.251227,-0.184477,-0.346207,-0.019513,-0.165865
precipitacion_arauca,0.034924,-0.066088,0.314739,1.000000,0.017710,0.045715,0.247388,0.264242,0.109682,0.209292,0.208583,0.072115,0.061164,-0.017620,0.275258,0.084851,0.107836,0.117684,0.025013,0.039089,0.109131,0.247661,-0.007031,-0.047415,0.056463,0.031036,0.096886,0.142209,0.191792,0.190068,0.130386,0.192756,0.130980,0.287361,0.038514,0.006995,0.380051,0.185561,-0.041487,-0.123384,-0.046943,-0.135892,-0.090326,-0.208100,-0.149469,0.031148,-0.063089,-0.109112,-0.107389,-0.158830,-0.071209,-0.242535,-0.054332,0.160982,0.006801,-0.131026,0.084866,0.017726,-0.094691,-0.038200,0.000818,-0.051431,0.021874,0.072079,-0.021756,0.022685,-0.260864
precipitacion_atlantico,0.115467,-0.075014,0.084880,0.017710,1.000000,0.013352,0.119168,0.152109,0.085969,0.019529,0.052842,0.056464,

Se considera una alta correlación entre variables a valores superiores a 0.5 o menores a -0.5, esto puede causar multicolinealidad y afectar la eficiencia de la predicción de los modelos, por lo tanto, se van a analizar las variables que tienen esta alta correlación para realizar una selección de variables inicial.


In [29]:
partial_correlation_df = calculate_partial_correlation_df(scaled_df_merged)
partial_correlation_df

,Variable,partial_correlation,abs_partial_correlation
0,brent_value,-0.201266,0.201266
1,precipitacion_amazonas,0.025642,0.025642
2,precipitacion_antioquia,-0.023005,0.023005
3,precipitacion_arauca,-0.016477,0.016477
4,precipitacion_atlantico,-0.042564,0.042564
...,...,...,...
62,temp_SUCRE,-0.012152,0.012152
63,temp_TOLIMA,-0.035211,0.035211
64,temp_VALLE DEL CAUCA,0.193226,0.193226
65,temp_VICHADA,-0.028915,0.028915


In [30]:
columns_to_keep, _ = apply_partial_correlation_criteria({ 
    "correlation_matrix": exogen_data_df_standardized.corr(),
    "partial_correlation_df": partial_correlation_df,
    "excluded": ["precipitacion_antioquia"]
})
columns_to_keep

['precipitacion_norte de santander',
 'precipitacion_antioquia',
 'precipitacion_boyaca',
 'precipitacion_bolivar',
 'precipitacion_arauca',
 'precipitacion_choco',
 'precipitacion_cauca',
 'precipitacion_guainia',
 'precipitacion_putumayo',
 'temp_BOYACA',
 'precipitacion_amazonas',
 'precipitacion_huila',
 'precipitacion_cundinamarca',
 'temp_AMAZONAS',
 'precipitacion_guaviare',
 'precipitacion_cordoba',
 'precipitacion_la guajira',
 'TRM',
 'precipitacion_cesar',
 'precipitacion_vaupes',
 'temp_ARAUCA',
 'precipitacion_casanare',
 'precipitacion_vichada',
 'precipitacion_quindio',
 'temp_CUNDINAMARCA',
 'brent_value',
 'precipitacion_atlantico',
 'precipitacion_magdalena',
 'temp_ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA',
 'precipitacion_meta',
 'temp_CAQUETA',
 'temp_NARINO',
 'precipitacion_narino',
 'precipitacion_caqueta',
 'precipitacion_bogota',
 'precipitacion_san andres providencia']

In [31]:
df_without_correlation = exogen_data_df[columns_to_keep]
new_correlation_matrix = df_without_correlation.corr()
print(f"Correlation Matrix Original: {new_correlation_matrix.shape}")
new_correlation_matrix.style.background_gradient(cmap="coolwarm")

Correlation Matrix Original: (36, 36)


,precipitacion_norte de santander,precipitacion_antioquia,precipitacion_boyaca,precipitacion_bolivar,precipitacion_arauca,precipitacion_choco,precipitacion_cauca,precipitacion_guainia,precipitacion_putumayo,temp_BOYACA,precipitacion_amazonas,precipitacion_huila,precipitacion_cundinamarca,temp_AMAZONAS,precipitacion_guaviare,precipitacion_cordoba,precipitacion_la guajira,TRM,precipitacion_cesar,precipitacion_vaupes,temp_ARAUCA,precipitacion_casanare,precipitacion_vichada,precipitacion_quindio,temp_CUNDINAMARCA,brent_value,precipitacion_atlantico,precipitacion_magdalena,temp_ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA,precipitacion_meta,temp_CAQUETA,temp_NARINO,precipitacion_narino,precipitacion_caqueta,precipitacion_bogota,precipitacion_san andres providencia
precipitacion_norte de santander,1.000000,0.194504,0.127610,0.041052,-0.047415,0.027519,0.127603,-0.025390,0.009300,0.065491,0.046852,0.118526,0.098627,-0.011762,0.070243,0.286116,0.113244,-0.352825,0.230750,-0.009287,0.131767,-0.126998,-0.059519,0.284040,-0.222202,0.126432,0.053496,0.135904,-0.108762,0.055692,-0.004543,0.194092,0.302257,-0.000406,0.110124,-0.000136
precipitacion_antioquia,0.194504,1.000000,0.391595,0.324677,0.314739,0.003961,0.335611,0.074976,0.083891,-0.101192,-0.000581,0.286523,0.297180,-0.087057,0.351991,0.455286,0.140865,-0.165865,0.137814,0.164454,0.180829,0.116232,0.026048,0.395891,-0.303399,0.329614,0.084880,0.404900,0.004907,0.345804,-0.220491,0.146918,0.351733,0.209265,0.270766,0.141533
precipitacion_boyaca,0.127610,0.391595,1.000000,0.202172,0.264242,0.213239,0.213500,0.097191,0.108749,-0.098030,0.003890,0.329143,0.338042,0.058891,0.317155,0.266711,0.153622,-0.072380,0.074206,-0.013271,0.058463,0.469496,0.202611,0.234871,-0.134366,0.051210,0.152109,0.276858,0.137253,0.262295,-0.129683,0.064011,0.204066,0.200954,0.330263,0.092542
precipitacion_bolivar,0.041052,0.324677,0.202172,1.000000,0.247388,-0.004278,0.109791,0.073121,-0.079292,-0.113995,-0.033369,0.018195,0.063309,0.007794,0.167940,0.338383,0.138250,-0.166742,0.142740,0.145462,0.158988,0.091918,0.047074,0.048677,-0.173457,0.191135,0.119168,0.235722,0.003807,0.089413,-0.077080,0.043375,0.090713,0.063806,0.074195,0.120488
precipitacion_arauca,-0.047415,0.314739,0.264242,0.247388,1.000000,-0.017620,0.072115,0.107836,0.056463,-0.135892,-0.066088,0.025013,0.084851,0.038514,0.117684,0.275258,0.039089,-0.260864,0.061164,0.130980,0.380051,0.208583,0.287361,0.031036,-0.158830,0.034924,0.017710,0.109131,0.185561,0.247661,-0.208100,0.084866,-0.007031,0.209292,0.045715,0.142209
precipitacion_choco,0.027519,0.003961,0.213239,-0.004278,-0.017620,1.000000,0.128701,-0.007055,0.134876,0.090258,0.083052,0.067838,0.103832,0.269572,0.170620,0.191771,-0.121592,-0.079713,-0.048245,0.263950,0.024089,0.198906,0.032960,0.057048,0.210660,-0.312554,0.085676,0.071512,0.120379,-0.136208,0.173642,0.196236,0.063042,-0.036104,-0.002043,0.059913
precipitacion_cauca,0.127603,0.335611,0.213500,0.109791,0.072115,0.128701,1.000000,-0.070319,0.094615,0.067971,0.087176,0.430154,0.290584,-0.090115,0.291185,0.159270,0.097752,0.118870,0.018856,0.203663,0.227512,0.027785,-0.094415,0.354683,-0.182039,0.036725,0.056464,0.320630,-0.086548,0.137821,0.080221,0.175159,0.472071,0.054377,0.181343,0.017864
precipitacion_guainia,-0.025390,0.074976,0.097191,0.073121,0.107836,-0.007055,-0.070319,1.000000,0.055070,-0.100795,-0.059546,0.060888,0.065340,-0.060151,0.114822,0.070223,0.000670,-0.082259,0.095596,0.040606,0.050947,0.141937,0.064548,-0.036821,-0.064620,0.134976,0.116575,0.037041,0.015437,0.134029,-0.151650,-0.018777,-0.005481,0.146477,0.059264,0.113545
precipitacion_putumayo,0.009300,0.083891,0.108749,-0.079292,0.056463,0.134876,0.094615,0.055070,1.000000,0.040466,-0.000844,0.331619,0.086850,-0.054078,0.112979,-0.022697,-0.019715,0.035349,0.006276,0.137690,-0.043160,0.155397,0.111173,0.066086,-0.014973,-0.153138,0.012250,-0.035423,0.013209,0.161010,-0.176497,-0.145711,0.221635,0.299014,0.033521,0.062644
temp_BOYACA,0.065491,

In [32]:
# exogen_data["Date"] = df_merged["Date"]
# exogen_data["energy_price"] = df_merged["energy_price"]
# scaled_df.to_csv("processed_tables/merged_no_damns_standardized.csv", index=False)